In [ ]:
chess_dict = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0],
}

def translate(board): 
    pgn = board.epd()
    foo = []  
    pieces = pgn.split(" ", 1)[0]
    rows = pieces.split("/")
    for row in rows:
        foo2 = []  
        for thing in row:
            if thing.isdigit():
                for i in range(0, int(thing)):
                    foo2.append(chess_dict['.'])
            else:
                foo2.append(chess_dict[thing])
        foo.append(foo2)
    return foo

In [ ]:
from keras.layers import Conv2D,Conv2DTranspose
from keras.layers import Activation,BatchNormalization
from keras.layers import Concatenate, Input
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense,Flatten

def complex_eval(image_shape=(8,8,12)):
  def define_encoder_block(layer_in, n_filters, batchnorm=True):
    g = Conv2D(n_filters,kernel_size = (2,2))(layer_in)
    if batchnorm:
      g = BatchNormalization()(g, training=True)
    g = Activation('relu')(g)
    return g
  in_image = Input(shape=image_shape)
  e1 = define_encoder_block(in_image, 64, batchnorm=False)
  e2 = define_encoder_block(e1, 128)
  e3 = define_encoder_block(e2, 256)
  e4 = define_encoder_block(e3, 512)
  e5 = define_encoder_block(e4, 512)
  e6 = define_encoder_block(e5, 512)
  b = Conv2D(filters = 1, kernel_size = (2,2))(e6)
  out_image = b
  model = Model(in_image, out_image)
  return model

def simple_eval(image_shape = (8,8,12)):
  model = Sequential()
  model.add(Dense(10,input_shape = (8,8,12)))
  model.add(Dense(10,activation = 'relu'))
  model.add(Flatten())
  model.add(Dense(1))
  return model 

In [ ]:
import random
import numpy as np

def calculate_move(board,evaluation,epochs = 10,depth = 10):
    first_legal_moves = str(board.legal_moves)[38:-2].replace(',','').split()
    scores = [[0]] * len(first_legal_moves)
    for epoch in range(epochs):
        for first_move in range(len(first_legal_moves)):
            play_board = board.copy()
            play_board.push_san(first_legal_moves[first_move])
            for _ in range(depth):
                legal_moves = str(play_board.legal_moves)[38:-2].replace(',','').split()
                if legal_moves:
                    move = random.choice(legal_moves)
                    play_board.push_san(move)
                else:
                    break
            translated = np.array(translate(play_board.copy))
            scores[first_move] += evaluation(translated)
    return first_legal_moves[scores.index(max(scores))]

In [ ]:
import random
import numpy as np
import time 

def monte_carlo_algo(board,evaluation,epochs = 5,depth = 5):
    first_legal_moves = list(board.legal_moves)
    scores = np.ones(len(first_legal_moves))
    for epoch in range(epochs):

        for first_move in range(len(first_legal_moves)):
            play_board = board.copy()
            play_board.push(first_legal_moves[first_move])

            for _ in range(depth):
                legal_moves = list(play_board.legal_moves)
                if legal_moves:
                    move = random.choice(legal_moves)
                    play_board.push(move)
                else:
                    break
                
            translated = np.array(translate(play_board))
            scores[first_move] += evaluation(translated)
    idx = np.where(scores == max(scores))[0][0]
    return first_legal_moves[idx]

In [ ]:
import chess

def fitness(agents):
    for agent in range(len(agents)-1):
        game = []
        board = chess.Board()
        
        other_agents = agents
        player_2 = random.choice(agents)
        
        player_1_idx = agent
        player_2_idx = agents.index(player_2)
        
        player_1 = agents[agent]

        counter = 0
        print('Game Started between Agent',player_1_idx,'and Agent',player_2_idx)
        while counter < 100 and board.is_game_over() == False:
            model  = player_1.neural_network

            def evaluation(input):
                pred = model(input.reshape(1,8,8,12))
                return pred

            move =  monte_carlo_algo(board,evaluation,epochs = 5,depth = 5)
            game.append(move)
            
            model = player_2.neural_network
            
            move =  monte_carlo_algo(board,evaluation,epochs = 5,depth = 5)
            game.append(move)
            counter += 1

        agents[player_1_idx].game = game
        agents[player_2_idx].game = game

        if board.is_checkmate:
            if counter % 2 == 0:
                agents[player_1_idx].fitness *= 1.5
                agents[player_2_idx].fitness *= 0.8
            else:
                agents[player_2_idx].fitness *= 1.5
                agents[player_1_idx].fitness *= 0.8
    return agents


In [ ]:
import numpy as np
import random
import numpy as np
from IPython.display import clear_output
from keras.models import clone_model
import tensorflow as tf
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,Flatten

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

def sigmoid(x):
    return 1/(1+np.exp(-x))

class genetic_algorithm:
        
    def execute(self,fitness,model,pop_size = 10,generations = 100,threshold = 1000):
        class Agent:
            def __init__(self,model):
                self.neural_network = clone_model(model)
                self.fitness = 100
                self.game = None
            def apply_weights(self,weights):
                self.neural_network.set_weights(weights)
            def __str__(self):
                    return 'Loss: ' + str(self.fitness)
                
        
                
        def generate_agents(population, network):
            return [Agent(model) for _ in range(population)]
        
        def selection(agents):
            agents = sorted(agents, key=lambda agent: agent.fitness, reverse=True)
            print('\n'.join(map(str, agents)))
            agents = agents[:int(0.2 * len(agents))]
            return agents
        
        def unflatten(flattened,shapes):
            newarray = []
            index = 0
            for shape in shapes:
                size = np.product(shape)
                newarray.append(flattened[index : index + size].reshape(shape))
                index += size
            return np.array(newarray)
        
        def crossover(agents,network,pop_size):
            offspring = []
            for _ in range((pop_size - len(agents)) // 2):
                parent1 = random.choice(agents)
                parent2 = random.choice(agents)
                child1 = Agent(network)
                child2 = Agent(network)
                
                
                shapes = [a.shape for a in parent1.neural_network.get_weights()]
                genes1 = np.concatenate([a.flatten() for a in parent1.neural_network.get_weights()])
                genes2 = np.concatenate([a.flatten() for a in parent2.neural_network.get_weights()])
                split = random.randint(0,len(genes1)-1)

                child1_genes = np.array(genes1[0:split].tolist() + genes2[split:].tolist())
                child2_genes = np.array(genes1[0:split].tolist() + genes2[split:].tolist())
                child1_genes = unflatten(child1_genes,shapes)
                child2_genes = unflatten(child2_genes,shapes)
            
                
                child1.apply_weights(list(child1_genes))
                child2.apply_weights(list(child2_genes))
                
                
                offspring.append(child1)
                offspring.append(child2)
            agents.extend(offspring)
            return agents
        
        def mutation(agents):
            for agent in agents:
                if random.uniform(0.0, 1.0) <= 0.1:
                    weights = agent.neural_network.get_weights()
                    shapes = [a.shape for a in weights]

                    flattened = np.concatenate([a.flatten() for a in weights])
                    randint = random.randint(0,len(flattened)-1)
                    flattened[randint] = np.random.randn()

                    newarray = unflatten(flattened,shapes)
                    agent.apply_weights(newarray)
            return agents
        
        loss = []
        for i in range(generations):
            print('Generation',str(i),':')
            agents = generate_agents(pop_size,model)
            agents = fitness(agents)
            agents = selection(agents)
            agents = crossover(agents,model,pop_size)
            agents = mutation(agents)
            agents = fitness(agents)
            loss.append(agents[0].fitness)
            if any(agent.fitness > threshold for agent in agents):
                print('Threshold met at generation '+str(i)+' !')
                
            if i % 100:
                clear_output()
                
        return agents[0],

In [ ]:
eval_model = simple_eval()
tourno_fitness = fitness

ga = genetic_algorithm()
agent,loss = ga.execute(tourno_fitness,eval_model,pop_size = 10,generations = 10)
print(agent.fitness)

board = chess.Board()

for move in agent.game:
    board.push_san(move)
print(board)
print(board.is_checkmate())
print(agent.game)